In [3]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [1]:
from langchain.embeddings import CacheBackedEmbeddings

In [5]:
from langchain.storage import LocalFileStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter

# OpenAI 임베딩을 사용하여 기본 임베딩 설정
embedding = HuggingFaceEmbeddings()

# 로컬 파일 저장소 설정
store = LocalFileStore("./cache/")

# 캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embedding,
    store,
    namespace="llama3",  # 기본 임베딩과 저장소를 사용하여 캐시 지원 임베딩을 생성
)

/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
list(store.yield_keys())

[]

In [7]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

# 문서 로드
raw_documents = TextLoader("./data/appendix-keywords.txt").load()
# 문자 단위로 텍스트 분할 설정
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# 문서 분할
documents = text_splitter.split_documents(raw_documents)

In [8]:
%time db = FAISS.from_documents(documents, cached_embedder)  # 문서로부터 FAISS 데이터베이스 생성

CPU times: user 922 ms, sys: 85.9 ms, total: 1.01 s
Wall time: 713 ms


In [9]:
%time db2 = FAISS.from_documents(documents, cached_embedder)

CPU times: user 5.92 ms, sys: 441 µs, total: 6.36 ms
Wall time: 5.61 ms


In [10]:
# VectorStore에서 처음 5개의 키를 가져옵니다.
list(store.yield_keys())[:5]

['llama39db9e1cd-62d8-50fc-94f4-24bef3cacaf5',
 'llama32112b0ec-6ade-59c9-b09c-755b33c3d32c',
 'llama30fd71f95-1342-512d-9d5b-3e3ab3c6bbe0',
 'llama3cc824f84-d691-544f-9d9c-ca7e45470bb2',
 'llama374ae75af-9058-555e-aefa-082f0b4e0560']

In [12]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import InMemoryByteStore

store = InMemoryByteStore()  # 메모리 내 바이트 저장소 생성

# 캐시 지원 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embedding, store, namespace="llama3"
)

In [16]:
%time db = FAISS.from_documents(documents, cached_embedder)  # 문서로부터 FAISS 데이터베이스 생성

CPU times: user 7.98 ms, sys: 28 µs, total: 8.01 ms
Wall time: 7.63 ms


In [17]:
list(store.yield_keys())

['llama341e7391b-b68f-5e9f-bb07-3609bb83c3e2',
 'llama3cc824f84-d691-544f-9d9c-ca7e45470bb2',
 'llama30fd71f95-1342-512d-9d5b-3e3ab3c6bbe0',
 'llama32112b0ec-6ade-59c9-b09c-755b33c3d32c',
 'llama37494a7c8-3399-52a1-85ef-f4d0a563d31f',
 'llama39db9e1cd-62d8-50fc-94f4-24bef3cacaf5',
 'llama374ae75af-9058-555e-aefa-082f0b4e0560']